<a href="https://colab.research.google.com/github/vmjaramillo1/stable_diffusion_presonal_train/blob/main/Fast_DreamBooth_Adaptaci%C3%B3n_DotCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#⭐ **DreamBooth colab From https://github.com/TheLastBen/fast-stable-diffusion**
###🛠️ Notebook adaptado por [@dotcsv](https://www.youtube.com/channel/UCy5znSnfMsDwaLlROnZ7Qbg). Actualizado a Python 3.8 por [@Deus;Gate](https://www.youtube.com/channel/UCNojH61Q5RcbBWSLW1UjTkw)

🔴 ESTE NOTEBOOK **ESTÁ TEMPORALMENTE FUERA DE SERVICIO** 🔴

Próximamente volverá a funcionar. Mientras tanto, cualquiera de las siguientes alternativas podrían servir para el entrenamiento de **Dreambooth** (*versión original usada en el tutorial*) o **LoRA** (*versión más ligera*).

| Notebook Name | Description | Link |
| --- | --- | --- |
| [Kohya LoRA Dreambooth](https://github.com/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb) | LoRA Training (Dreambooth method) | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=for-the-badge)](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb) |
| [Kohya LoRA Fine-Tuning](https://github.com/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-finetuner.ipynb) | LoRA Training (Fine-tune method) | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=for-the-badge)](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-finetuner.ipynb) |
| [Kohya Trainer](https://github.com/Linaqruf/kohya-trainer/blob/main/kohya-trainer.ipynb) | Native Training | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=for-the-badge)](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-trainer.ipynb) |
| [Kohya Dreambooth](https://github.com/Linaqruf/kohya-trainer/blob/main/kohya-dreambooth.ipynb) | Dreambooth Training | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=for-the-badge)](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-dreambooth.ipynb) |

🚨 Ten activada la **Aceleración por hardware** con GPU en `"Entorno de ejecución" > "Cambiar tipo de entorno de ejecución"`

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a11feed7-c321-6344-4776-2ee7dde337e0)


### **Paso 1** - Conectamos con Google Drive. **Importante contar con unos 4GB de almacenamiento.**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### **Paso 2** - Instalamos las librerías necesarias.

In [ ]:
#@markdown # Dependencies

from IPython.utils import capture
import time

print('[1;32mInstalling dependencies...')
with capture.capture_output() as cap:
    %cd /content/
    !pip install -q --no-deps accelerate==0.12.0
    !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/db.txt
    !dpkg -i *.deb
    !tar -C / --zstd -xf db_deps.tar.zst
    !rm *.deb | rm *.zst | rm *.txt
    !git clone --depth 1 --branch updt https://github.com/TheLastBen/diffusers

print('[1;32mDone, proceed')

### **Paso 3** - Descargamos el modelo .ckpt de Stable Diffusion original.


```
🔴 ¡UPDATE! Ya no es necesario introducir tu token de Hugging Face 😊
```

In [ ]:
import os
import time
from IPython.display import clear_output
import wget

#@markdown ---

#@markdown <b>Elige el modelo a utilizar:</b> 

#@markdown Ahora puedes usar la técnica de DreamBooth incluso en versiones más actualizadas de Stable Diffusion: La versión 1.5 te ofrece un estilo más artístico (<i>es la utilizada en el tutorial de Youtube</i>). Y las versiones 2.1 servirán como una mejor base para obtener imágenes más fotorrealistas.

#@markdown 🚧 <i>Temporalmente el soporte a la v2.1 estará deshabilitado. 🚧

Model_Version = "1.5" #@param [ "1.5"]

#@markdown ---

with capture.capture_output() as cap: 
  %cd /content/

if os.path.exists('/content/gdrive/MyDrive/Fast-Dreambooth/token.txt'):
  with open("/content/gdrive/MyDrive/Fast-Dreambooth/token.txt") as f:
     token = f.read()
  authe=f'https://USER:{token}@'
else:
  authe="https://"

def downloadmodel():

  if os.path.exists('/content/stable-diffusion-v1-5'):
    !rm -r /content/stable-diffusion-v1-5
  clear_output()

  %cd /content/
  clear_output()
  !mkdir /content/stable-diffusion-v1-5
  %cd /content/stable-diffusion-v1-5
  !git init
  !git lfs install --system --skip-repo
  !git remote add -f origin  "https://huggingface.co/runwayml/stable-diffusion-v1-5"
  !git config core.sparsecheckout true
  !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
  !git pull origin main
  if os.path.exists('/content/stable-diffusion-v1-5/unet/diffusion_pytorch_model.bin'):
    !git clone "https://huggingface.co/stabilityai/sd-vae-ft-mse"
    !mv /content/stable-diffusion-v1-5/sd-vae-ft-mse /content/stable-diffusion-v1-5/vae
    !rm -r /content/stable-diffusion-v1-5/.git
    %cd /content/stable-diffusion-v1-5
    !rm model_index.json
    time.sleep(1)    
    wget.download('https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/model_index.json')
    !sed -i 's@"clip_sample": false@@g' /content/stable-diffusion-v1-5/scheduler/scheduler_config.json
    !sed -i 's@"trained_betas": null,@"trained_betas": null@g' /content/stable-diffusion-v1-5/scheduler/scheduler_config.json
    !sed -i 's@"sample_size": 256,@"sample_size": 512,@g' /content/stable-diffusion-v1-5/vae/config.json  
    %cd /content/
    clear_output()
    print('[1;32mDONE !')
  else:
    while not os.path.exists('/content/stable-diffusion-v1-5/unet/diffusion_pytorch_model.bin'):
         print('[1;31mSomething went wrong')
         time.sleep(5)

def newdownloadmodel():

  %cd /content/
  clear_output()
  !mkdir /content/stable-diffusion-v2-768
  %cd /content/stable-diffusion-v2-768
  !git init
  !git lfs install --system --skip-repo
  !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1"
  !git config core.sparsecheckout true
  !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
  !git pull origin main
  !rm -r /content/stable-diffusion-v2-768/.git
  clear_output()
  print('[1;32mDONE !')


def newdownloadmodelb():

  %cd /content/
  clear_output()
  !mkdir /content/stable-diffusion-v2-512
  %cd /content/stable-diffusion-v2-512
  !git init
  !git lfs install --system --skip-repo
  !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1-base"
  !git config core.sparsecheckout true
  !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
  !git pull origin main
  !rm -r /content/stable-diffusion-v2-512/.git
  clear_output()
  print('[1;32mDONE !')


if Model_Version=="1.5":
  if not os.path.exists('/content/stable-diffusion-v1-5'):
    downloadmodel()
    MODEL_NAME="/content/stable-diffusion-v1-5"
  else:
    MODEL_NAME="/content/stable-diffusion-v1-5"
    print("[1;32mThe v1.5 model already exists, using this model.")
elif Model_Version=="V2.1-512px":
  if not os.path.exists('/content/stable-diffusion-v2-512'):
    newdownloadmodelb()
    MODEL_NAME="/content/stable-diffusion-v2-512"
  else:
    MODEL_NAME="/content/stable-diffusion-v2-512"
    print("[1;32mThe v2-512px model already exists, using this model.")
elif Model_Version=="V2.1-768px":
  if not os.path.exists('/content/stable-diffusion-v2-768'):
    newdownloadmodel()
    MODEL_NAME="/content/stable-diffusion-v2-768"
  else:
    MODEL_NAME="/content/stable-diffusion-v2-768"
    print("[1;32mThe v2-768px model already exists, using this model.")

### **Paso 4** - Configuramos el entrenamiento de Dreambooth.

In [ ]:
import os
import wget
import time
import shutil
import ipywidgets as widgets

from IPython.display import clear_output
from IPython.utils import capture
from os import listdir
from os.path import isfile
from google.colab import files
from PIL import Image
from tqdm import tqdm
from io import BytesIO

import os
import shutil
from google.colab import files
from IPython.display import clear_output
from IPython.utils import capture

#@markdown ---
Training_Subject = "Character" #@param ["Character", "Object", "Style", "Artist", "Movie", "TV Show"] 

With_Prior_Preservation = "Yes" #@param ["Yes", "No"] 
#@markdown - With the prior reservation method, the results are better, you will either have to upload around 200 pictures of the class you're training (dog, person, car, house ...) or let Dreambooth generate them.

MODEL_NAME="/content/stable-diffusion-v1-5"

#@markdown ---

SUBJECT_TYPE = "person" #@param{type: 'string'}
while SUBJECT_TYPE=="":
   SUBJECT_TYPE=input('Input the subject type:')

#@markdown - If you're training on a character or an object, the subject type would be : Man, Woman, Shirt, Car, Dog, Baby ...etc
#@markdown - If you're training on a Style, the subject type would be : impressionist, brutalist, abstract, use "beautiful" for a general style...etc
#@markdown - If you're training on a Movie/Show, the subject type would be : Action, Drama, Science-fiction, Comedy ...etc
#@markdown - If you're training on an Artist, the subject type would be : Painting, sketch, drawing, photography, art ...etc

INSTANCE_NAME= "tu_token_especial" #@param{type: 'string'}
while INSTANCE_NAME=="":
   INSTANCE_NAME=input('Input the instance name (identifier) :')

#@markdown - The instance is an identifier, choose a unique identifier unknown by stable diffusion. 

INSTANCE_DIR_OPTIONAL="" #@param{type: 'string'}
INSTANCE_DIR=INSTANCE_DIR_OPTIONAL
while INSTANCE_DIR_OPTIONAL!="" and not os.path.exists(str(INSTANCE_DIR)):
    INSTANCE_DIR=input('[1;31mThe instance folder specified does not exist, use the colab file explorer to copy the path :')

#@markdown - If the number of instance pictures is large, it is preferable to specify directly the folder instead of uploading, leave EMPTY to upload.

CLASS_DIR="/content/data/"+ SUBJECT_TYPE
Number_of_subject_images=500#@param{type: 'number'}
while Number_of_subject_images==None:
     Number_of_subject_images=input('Input the number of subject images :')
SUBJECT_IMAGES=Number_of_subject_images

Save_class_images_to_gdrive = False #@param {type:"boolean"}
#@markdown - Save time in case you're training multiple instances of the same class

if Training_Subject=="Character" or Training_Subject=="Object":
  PT="photo of "+INSTANCE_NAME+" "+SUBJECT_TYPE
  CPT="a photo of a "+SUBJECT_TYPE+", ultra detailed"
  
OUTPUT_DIR="/content/models/"+ INSTANCE_NAME

if INSTANCE_DIR_OPTIONAL=="":
  INSTANCE_DIR="/content/data/"+INSTANCE_NAME
  !mkdir -p "$INSTANCE_DIR"
  uploaded = files.upload()
  for filename in uploaded.keys():
    shutil.move(filename, INSTANCE_DIR)
    clear_output()

with capture.capture_output() as cap:
   %cd "$INSTANCE_DIR"
   !find . -name "* *" -type f | rename 's/ /_/g'
   %cd /content
print('[1;32mOK')

try:
  MODEL_NAME
  pass
except:
  MODEL_NAME=""
  
PT=SUBJECT_TYPE

Session_Name = INSTANCE_NAME

WORKSPACE='/content/gdrive/MyDrive/Fast-Dreambooth'

Session_Link_optional = "" # Not used

INSTANCE_NAME=Session_Name
OUTPUT_DIR="/content/models/"+Session_Name
SESSION_DIR=WORKSPACE+'/Sessions/'+Session_Name
INSTANCE_DIR=SESSION_DIR+'/instance_images'
CONCEPT_DIR=SESSION_DIR+'/concept_images'
CAPTIONS_DIR=SESSION_DIR+'/captions'
MDLPTH=str(SESSION_DIR+"/"+Session_Name+'.ckpt')

%cd /content

### **Paso 5** - (Opcional) Descargamos imágenes de regularización.  💖 Gracias [Joe Penna](https://github.com/JoePenna/Dreambooth-Stable-Diffusion)!

In [ ]:
#@markdown We’ve created the following image sets
#@markdown - `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
#@markdown - `man_unsplash` - pictures from various photographers
#@markdown - `person_ddim`
#@markdown - `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0 <br />
#@markdown - `blonde_woman` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0 <br />

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "blonde_woman"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

if not os.path.exists(str(CONCEPT_DIR)):
  %mkdir -p "$CONCEPT_DIR"

%mkdir -p regularization_images/{dataset}
%mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* "$CONCEPT_DIR"
CLASS_DIR=CONCEPT_DIR

### **Paso 6** - ...y ahora **¡A ENTRENAR!** 💪

In [ ]:
#@markdown ---
#@markdown #Start DreamBooth
#@markdown ---
import os
from subprocess import getoutput
from IPython.display import clear_output
from google.colab import runtime
import time
import random

import os
import shutil


def rename_images(src_dir, dest_dir, token):
    # Eliminar contenido del directorio de destino
    for file_name in os.listdir(dest_dir):
        file_path = os.path.join(dest_dir, file_name)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f'Error al eliminar {file_name} en {dest_dir}. Error: {e}')
            return
    
    # Obtener todos los archivos de imagen en el directorio de origen
    src_files = [f for f in os.listdir(src_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.gif'))]
    
    # Crear el directorio de destino si no existe
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    # Recorrer todos los archivos de imagen y renombrarlos
    for i, src_file in enumerate(src_files):
        # Construir el nuevo nombre del archivo
        if i == 0:
            new_name = f'{token}.{src_file.split(".")[-1]}'
        else:
            new_name = f'{token}-({i}).{src_file.split(".")[-1]}'
        
        # Construir las rutas completas
        src_path = os.path.join(src_dir, src_file)
        dest_path = os.path.join(dest_dir, new_name)
                
        # Renombrar y mover el archivo
        shutil.copy2(src_path, dest_path)


if not os.path.exists(INSTANCE_DIR):
    # Crear el directorio si no existe
    os.makedirs(INSTANCE_DIR)    

if not os.path.exists(CAPTIONS_DIR):
    # Crear el directorio si no existe
    os.makedirs(CAPTIONS_DIR)

rename_images("/content/data/" + INSTANCE_NAME, INSTANCE_DIR, INSTANCE_NAME)

if os.path.exists(INSTANCE_DIR+"/.ipynb_checkpoints"):
  %rm -r $INSTANCE_DIR"/.ipynb_checkpoints"

if os.path.exists(CONCEPT_DIR+"/.ipynb_checkpoints"):
  %rm -r $CONCEPT_DIR"/.ipynb_checkpoints"

if os.path.exists(CAPTIONS_DIR+"/.ipynb_checkpoints"):
  %rm -r $CAPTIONS_DIR"/.ipynb_checkpoints"

if os.path.exists(CAPTIONS_DIR+"off"):
  !mv $CAPTIONS_DIR"off" $CAPTIONS_DIR

MODELT_NAME=MODEL_NAME

UNet_Training_Steps=3000 #@param{type: 'number'}
UNet_Learning_Rate = 5e-6 #@param ["2e-5","1e-5","9e-6","8e-6","7e-6","6e-6","5e-6", "4e-6", "3e-6", "2e-6"] {type:"raw"}
untlr=UNet_Learning_Rate

Text_Encoder_Training_Steps=250

Text_Encoder_Concept_Training_Steps=0

Text_Encoder_Learning_Rate = 1e-6
txlr=Text_Encoder_Learning_Rate

trnonltxt=""
if UNet_Training_Steps==0:
   trnonltxt="--train_only_text_encoder"

Seed=''

External_Captions = False

extrnlcptn=""
if External_Captions:
  extrnlcptn="--external_captions"


Style_Training = False

Style=""
if Style_Training:
  Style="--Style"

Resolution = "512"
Res=int(Resolution)

fp16 = True

if Seed =='' or Seed=='0':
  Seed=random.randint(1, 999999)
else:
  Seed=int(Seed)

GC="--gradient_checkpointing"

if fp16:
  prec="fp16"
else:
  prec="no"

s = getoutput('nvidia-smi')
if 'A100' in s:
  GC=""

precision=prec

resuming=""

Enable_text_encoder_training= True
Enable_Text_Encoder_Concept_Training= True

if Text_Encoder_Training_Steps==0 or External_Captions:
   Enable_text_encoder_training= False
else:
  stptxt=Text_Encoder_Training_Steps

if Text_Encoder_Concept_Training_Steps==0:
   Enable_Text_Encoder_Concept_Training= False
else:
  stptxtc=Text_Encoder_Concept_Training_Steps

Save_Checkpoint_Every_n_Steps = False #@param {type:"boolean"}
Save_Checkpoint_Every=500
if Save_Checkpoint_Every==None:
  Save_Checkpoint_Every=1

stp=0
Start_saving_from_the_step=500
if Start_saving_from_the_step==None:
  Start_saving_from_the_step=0
if (Start_saving_from_the_step < 200):
  Start_saving_from_the_step=Save_Checkpoint_Every
stpsv=Start_saving_from_the_step
if Save_Checkpoint_Every_n_Steps:
  stp=Save_Checkpoint_Every
#@markdown - Start saving intermediary checkpoints from this step.

Disconnect_after_training=False #@param {type:"boolean"}

#@markdown - Auto-disconnect from google colab after the training to avoid wasting compute units.

def dump_only_textenc(trnonltxt, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, precision, Training_Steps):
    
    !accelerate launch /content/diffusers/examples/dreambooth/train_dreambooth.py \
    $trnonltxt \
    --image_captions_filename \
    --train_text_encoder \
    --dump_only_text_encoder \
    --pretrained_model_name_or_path="$MODELT_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --instance_prompt="$PT" \
    --seed=$Seed \
    --resolution=512 \
    --mixed_precision=$precision \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 $GC \
    --use_8bit_adam \
    --learning_rate=$txlr \
    --lr_scheduler="linear" \
    --lr_warmup_steps=0 \
    --max_train_steps=$Training_Steps

def train_only_unet(stpsv, stp, SESSION_DIR, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, Res, precision, Training_Steps):
    clear_output()
    if resuming=="Yes":
      print('[1;32mResuming Training...[0m')
    print('[1;33mTraining the UNet...[0m')
    !accelerate launch /content/diffusers/examples/dreambooth/train_dreambooth.py \
    $Style \
    $extrnlcptn \
    --stop_text_encoder_training=$Text_Encoder_Training_Steps \
    --image_captions_filename \
    --train_only_unet \
    --save_starting_step=$stpsv \
    --save_n_steps=$stp \
    --Session_dir=$SESSION_DIR \
    --pretrained_model_name_or_path="$MODELT_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --captions_dir="$CAPTIONS_DIR" \
    --instance_prompt="$PT" \
    --seed=$Seed \
    --resolution=$Res \
    --mixed_precision=$precision \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 $GC \
    --use_8bit_adam \
    --learning_rate=$untlr \
    --lr_scheduler="linear" \
    --lr_warmup_steps=0 \
    --max_train_steps=$Training_Steps


if Enable_text_encoder_training :
  print('[1;33mTraining the text encoder...[0m')
  if os.path.exists(OUTPUT_DIR+'/'+'text_encoder_trained'):
    %rm -r $OUTPUT_DIR"/text_encoder_trained"
  dump_only_textenc(trnonltxt, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, precision, Training_Steps=stptxt)

if Enable_Text_Encoder_Concept_Training:
  if os.path.exists(CONCEPT_DIR):
    if os.listdir(CONCEPT_DIR)!=[]:
      clear_output()
      if resuming=="Yes":
        print('[1;32mResuming Training...[0m')
      print('[1;33mTraining the text encoder on the concept...[0m')
      dump_only_textenc(trnonltxt, MODELT_NAME, CONCEPT_DIR, OUTPUT_DIR, PT, Seed, precision, Training_Steps=stptxtc)
    else:
      clear_output()
      if resuming=="Yes":
        print('[1;32mResuming Training...[0m')
      print('[1;31mNo concept images found, skipping concept training...')
      Text_Encoder_Concept_Training_Steps=0
      time.sleep(8)
  else:
      clear_output()
      if resuming=="Yes":
        print('[1;32mResuming Training...[0m')
      print('[1;31mNo concept images found, skipping concept training...')
      Text_Encoder_Concept_Training_Steps=0
      time.sleep(8)

if UNet_Training_Steps!=0:
  train_only_unet(stpsv, stp, SESSION_DIR, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, Res, precision, Training_Steps=UNet_Training_Steps)

if UNet_Training_Steps==0 and Text_Encoder_Concept_Training_Steps==0 and External_Captions :
  print('[1;32mNothing to do')
else:
  if os.path.exists('/content/models/'+INSTANCE_NAME+'/unet/diffusion_pytorch_model.bin'):
    prc="--fp16" if precision=="fp16" else ""
    !python /content/diffusers/scripts/convertosdv2.py $prc $OUTPUT_DIR $SESSION_DIR/$Session_Name".ckpt"
    clear_output()
    if os.path.exists(SESSION_DIR+"/"+INSTANCE_NAME+'.ckpt'):
      clear_output()
      print("[1;32mDONE, the CKPT model is in your Gdrive in the sessions folder")
      if Disconnect_after_training :
        time.sleep(20)
        runtime.unassign()
    else:
      print("[1;31mSomething went wrong")
  else:
    print("[1;31mSomething went wrong")

### **Paso 7** (Opcional) - **Prueba el modelo**


In [ ]:
import os
import time
import sys
import fileinput
from IPython.display import clear_output
from subprocess import getoutput
from IPython.utils import capture

#@markdown <h3> ⬅️ ¡Ejecútame y suerte! 🤞 </h3> 


Model_Version = "1.5"
Previous_Session=""

Use_Custom_Path = False

try:
  INSTANCE_NAME
  INSTANCET=INSTANCE_NAME
except:
  pass

if Previous_Session!="":
  INSTANCET=Previous_Session
  INSTANCET=INSTANCET.replace(" ","_")

if Use_Custom_Path:
  try:
    INSTANCET
    del INSTANCET
  except:
    pass

try:
  INSTANCET
  if Previous_Session!="":
    path_to_trained_model='/content/gdrive/MyDrive/Fast-Dreambooth/Sessions/'+Previous_Session+"/"+Previous_Session+'.ckpt'
  else:
    path_to_trained_model=SESSION_DIR+"/"+INSTANCET+'.ckpt'
except:
  print('[1;31mIt seems that you did not perform training during this session [1;32mor you chose to use a custom path,\nprovide the full path to the model (including the name of the model):\n')
  path_to_trained_model=input()
     
while not os.path.exists(path_to_trained_model):
   print("[1;31mThe model doesn't exist on you Gdrive, use the file explorer to get the path : ")
   path_to_trained_model=input()
   
fgitclone = "git clone --depth 1"

with capture.capture_output() as cap:
    if not os.path.exists('/content/gdrive/MyDrive/Fast-Dreambooth'):
      !mkdir -p /content/gdrive/MyDrive/Fast-Dreambooth
    time.sleep(2)
    %mkdir -p /content/gdrive/MyDrive/Fast-Dreambooth/sd_db
    %cd /content/gdrive/MyDrive/Fast-Dreambooth/sd_db
    !$fgitclone --branch main https://github.com/Stability-AI/stablediffusion
    !$fgitclone --branch Colabdb https://github.com/TheLastBen/stable-diffusion-webui
    !mkdir -p /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stable-diffusion-webui/cache/huggingface
    !ln -s /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stable-diffusion-webui/cache/huggingface /root/.cache/
    %cd /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stable-diffusion-webui

clear_output()
print('[1;32m')
!git pull

with capture.capture_output() as cap:
  if not os.path.exists('/content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stablediffusion/src/k-diffusion/k_diffusion'):
    !mkdir /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stablediffusion/src
    %cd /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stablediffusion/src
    !$fgitclone https://github.com/TheLastBen/taming-transformers
    !$fgitclone https://github.com/salesforce/BLIP blip
    !$fgitclone https://github.com/sczhou/CodeFormer codeformer
    !$fgitclone --branch master https://github.com/crowsonkb/k-diffusion

with capture.capture_output() as cap:
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel

Use_localtunnel = False

User = ""
Password= ""

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

share=''
if not Use_localtunnel:
  share='--share'
  !wget -q -O /usr/local/lib/python3.8/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/blocks.py

else:
  with capture.capture_output() as cap:
    share=''
    %cd /content
    !nohup lt --port 7860 > srv.txt 2>&1 &
    time.sleep(2)
    !grep -o 'https[^ ]*' /content/srv.txt >srvr.txt
    time.sleep(2)
    srv= getoutput('cat /content/srvr.txt')

    for line in fileinput.input('/usr/local/lib/python3.8/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)

    !rm /content/srv.txt
    !rm /content/srvr.txt
    %cd /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stable-diffusion-webui

if Model_Version == "V2.1-768":
  configf="--config /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stablediffusion/configs/stable-diffusion/v2-inference-v.yaml"
elif Model_Version == "V2.1-512":
  configf="--config /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stablediffusion/configs/stable-diffusion/v2-inference.yaml"
else:
  configf=""

clear_output()

if os.path.isfile(path_to_trained_model):
  !python /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stable-diffusion-webui/webui.py $share --ckpt "$path_to_trained_model" $configf $auth
else:
  !python /content/gdrive/MyDrive/Fast-Dreambooth/sd_db/stable-diffusion-webui/webui.py $share --ckpt-dir "$path_to_trained_model" $configf $auth